In [ ]:
!pip install PyMySQL
!pip install peewee

In [13]:
#  connection on EC2 RDS

import peewee
import pymysql

db = peewee.MySQLDatabase('myproject',
                          user='BB1032',
                          host='localhost',
                          password='IamBB1032',
                          port=5200)

In [23]:
# connection on localhost
# Setting database's connection

import peewee
import pymysql

db = peewee.MySQLDatabase('myproject',
                          user='root',
                          host='localhost',
                          password='athena1225')

In [14]:
# connected successfully
# must bulid the schema before connected


db.get_columns('brand')

[ColumnMetadata(name='brand_id', data_type='int', null=False, primary_key=True, table='brand'),
 ColumnMetadata(name='brand_name', data_type='varchar', null=True, primary_key=False, table='brand')]

In [15]:
class brand(peewee.Model):
    brand_id = peewee.IntegerField()
    brand_name = peewee.CharField()
    
    class Meta:
        database = db

In [4]:
# Functions

def brand_numeric(uni_file):
    
    # making a clean brand list
    
    brand_lists = []
    brand_lists.append(uni_file['brand'])
    
    unique_brand_lists = []
    
    for brand in brand_lists:
        if brand not in unique_brand_lists :
            unique_brand_lists.append(brand)
            
        
    brands = []

    for i in unique_brand_lists:
        bb = {'%s'%i : unique_brand_lists.index(i)}
        #print(bb)
        brands.append(bb)
    
    
    # changing file's brand into numeric
    
    for x in range(len(brands)) :
        if uni_file['brand'] not in brands[x].keys() :
            continue              
        else :
            br_num = brands.index({'%s'%uni_file['brand']:x})  
            values = brands[br_num]['%s'%uni_file['brand']]
            #print(values)
            uni_file['brand'] = values
            #print(uni_file)
        
    return uni_file


In [5]:
# Function

def unified_field(j_file):
    
    if 'listing_date' not in j_file.keys():
        j_file['listing_date']='null'
            
            #pprint(j_file)
    if 'product_color' not in j_file.keys():
        j_file['product_color']='one_color'
            
    if 'product_ID'not in j_file.keys():
        j_file['product_ID']='null'
            
    if 'product_name'not in j_file.keys():
        j_file['product_name']='null'
            
    if 'brand'not in j_file.keys():
        j_file['brand']='null'
            
    if 'price'not in j_file.keys():
        j_file['price']='null'
            
    if 'volumn'not in j_file.keys():
        j_file['volumn']='null'
    
    if 'score'not in j_file.keys():
        j_file['score']= 0

    
    if 'pro_picture'not in j_file.keys():
        j_file['pro_picture']='null'

    
    if 'pro_class'not in j_file.keys():
        j_file['pro_class']='%s'%reg_class
        
        
    return j_file

In [6]:
# Function

def class_into_num(class_):
    
    if class_ == 'eyebrow' :
        class_ = '1'
    elif class_ == 'eye_liner':
        class_ = '2'
    elif class_ == 'base':
        class_ = '3'
    elif class_ == 'lip_stick':
        class_ = '4'
    else:
        print("out of classification")
        
    return class_

In [7]:
# Making a brand_lists(duplicated content)

import glob
import json
import codecs
from pprint import pprint

files_base = glob.glob('D:\\project\\urcosme\\eyebrow\\*.json')


dir_lists = ['eyebrow', 'base', 'eye_liner', 'lip_stick']

brand_lists = []

for dir_list in dir_lists:
    
    files_base = glob.glob('D:\\project\\urcosme\\%s\\*.json'%dir_list)
    
    for i in files_base :
        
        class_ = i.split('\\')[3]
        
        reg_class = class_into_num(class_)
        
          
    
        with codecs.open(i,'r','utf-8')as file:
            
            j_file = json.load(file)
            
            uni_file = unified_field(j_file)
            
            brand_lists.append(uni_file['brand'])
        
        
     
    



In [8]:
# distinct brand list 

unique_brand_lists = []

for brand in brand_lists:
     if brand not in unique_brand_lists :
            unique_brand_lists.append(brand)
            
unique_brand_lists
len(unique_brand_lists)

376

In [20]:
# give brand_id

brands = []

for i in unique_brand_lists:
    bb = {'%s'%i : unique_brand_lists.index(i)}
    #print(bb)
    brands.append(bb)
    
#brands

In [16]:
# insert data into brand -> ok

in_brands = []


    
for i in unique_brand_lists :
    bb = {'brand_name': '%s'%i, 'brand_id': unique_brand_lists.index(i)}
    in_brands.append(bb)
    
#in_brands

brand_dict = ({'brand_id': in_brands[i]['brand_id'],
               'brand_name': in_brands[i]['brand_name']} for i in range(len(in_brands)))

with db.atomic():
    brand.insert_many(brand_dict).execute()